### Author: [Pratik Sharma](https://github.com/sharmapratik88/)
# Project 3 - Recurrent Neural Network - TV Script Generation

In this project, we'll generate our own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs. We'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons. The Neural Network we'll build will generate a new, "fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided in `./data/Seinfeld_Scripts.txt` and let's open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, we'll define and train an RNN to generate a new script!

In [1]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Setting the current working directory
import os; os.chdir('drive/My Drive/Udacity/Project03_TV Script Generation')

In [3]:
!nvidia-smi

Sat May 23 18:42:27 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# Import packages
import helper, problem_unittests as tests
import numpy as np, torch

from torch.utils.data import TensorDataset, DataLoader

# Set seed
seed = 2020
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

# Suppress warnings
import warnings; warnings.filterwarnings('ignore')

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu: print('No GPU found. Please use a GPU to train your neural network.')

In [0]:
# load in data

data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Playing around with `view_line_range` to view different parts of the data. This will give us a sense of the data we'll be working with. We can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [6]:
view_line_range = (0, 10)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [7]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    vocab = set(text)
    w2i = {w:i for i, w in enumerate(vocab)}
    i2w = {i:w for i, w in enumerate(vocab)}
    # return tuple
    return (w2i, i2w)

tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [8]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    sym = {'.': '||Period||', 
           ',': '||Comma||',
           '"': '||Quotation_Mark||',
           ';': '||Semicolon||', 
           '!': '||Exclamation_mark||',
           '?': '||Question_mark||',
           '(': '||Left_Parentheses||',
           ')': '||Right_Parentheses||',
           '-': '||Dash||',
           '\n': '||Return||'}
        
    return sym

tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. We can also look at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail.

In [0]:
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is our first checkpoint. If we ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [0]:
int_text, w2i, i2w, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, we'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

We can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, batch_size = batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>We can batch words using the DataLoader, but it will be up to us to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [11]:
def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    
    feature_tens, target_tens = [], []
    word_length = len(words)
    n_targets = word_length - sequence_length
    
    for x in range(0, n_targets):
        feature = words[x : sequence_length + x]
        target = words[sequence_length + x]
        feature_tens.append(feature); target_tens.append(target)
        
    feature_tens, target_tens = torch.from_numpy(np.asarray(feature_tens)), torch.from_numpy(np.asarray(target_tens))
    
    data = TensorDataset(feature_tens, target_tens)
    data_loader = DataLoader(data, batch_size = batch_size, shuffle = True)
    return data_loader

# Check for a sample words, sequence length and batch size
words, sequence_length, batch_size = range(10), 5, 1
data_loader = batch_data(words, sequence_length, batch_size)

print(data_loader.dataset.tensors)

(tensor([[0, 1, 2, 3, 4],
        [1, 2, 3, 4, 5],
        [2, 3, 4, 5, 6],
        [3, 4, 5, 6, 7],
        [4, 5, 6, 7, 8]]), tensor([5, 6, 7, 8, 9]))


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [12]:
# test dataloader

test_text = range(50); t_loader = batch_data(test_text, sequence_length = 5, batch_size = 10)

data_iter = iter(t_loader); sample_x, sample_y = data_iter.next()

print('Shape of feature sample: {}'.format(sample_x.shape))
print('Feature Sample:\n{}\n'.format(sample_x))

print('Shape of target sample: {}'.format(sample_y.shape))
print('Target Sample: {}'.format(sample_y))

Shape of feature sample: torch.Size([10, 5])
Feature Sample:
tensor([[42, 43, 44, 45, 46],
        [39, 40, 41, 42, 43],
        [44, 45, 46, 47, 48],
        [36, 37, 38, 39, 40],
        [34, 35, 36, 37, 38],
        [24, 25, 26, 27, 28],
        [17, 18, 19, 20, 21],
        [29, 30, 31, 32, 33],
        [15, 16, 17, 18, 19],
        [25, 26, 27, 28, 29]])

Shape of target sample: torch.Size([10])
Target Sample: tensor([47, 44, 49, 41, 39, 29, 22, 34, 20, 30])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). We can choose to use a GRU or an LSTM. To complete the RNN, implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [13]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout = 0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout = dropout, batch_first = True)
        
        self.dropout = nn.Dropout(dropout)
        
        self.fc = nn.Linear(hidden_dim,output_size)
        
        def _weights_init(m):
            if isinstance(m, nn.Conv2d or nn.Linear or nn.GRU or nn.LSTM):
                init.xavier_normal_(m.weight)
                m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d or nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        
        self.apply(_weights_init)
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        
        batch_size = nn_input.size(0)
        embeds = self.embedding(nn_input.long())
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        output = self.dropout(lstm_out)
        output = self.fc(output)
        output = output.view(batch_size, -1, self.output_size)
        out = output[:, -1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        """
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        """        
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [14]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
   
    # move data to GPU, if available
    if train_on_gpu: inp, target = inp.cuda(), target.cuda()
    hidden = tuple([each.data for each in hidden])
    optimizer.zero_grad()
    
    if train_on_gpu: inpt, tgt = inp.type(torch.cuda.FloatTensor).detach(), target.type(torch.cuda.LongTensor).detach()
    else: inpt, tgt = inp.type(torch.FloatTensor), target.type(torch.LongTensor)
    
    output, hidden = rnn(inpt, hidden)
    loss = criterion(output, tgt)
    
    # perform backpropagation and optimization
    loss.backward(retain_graph = True)
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    optimizer.step()
    
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter.

In [0]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches = 100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [0]:
# Data params

# Sequence Length
sequence_length = 25

# Batch Size
batch_size = 256

# Length of w2i dictionary
len_w2i = len(w2i)

# Data Loader
train_loader = batch_data(int_text, sequence_length, batch_size)

In [0]:
# Training parameters

# Number of Epochs
num_epochs = 10

# Learning Rate
learning_rate = 0.001 

# Model parameters

# Vocab size
vocab_size = len(i2w)

# Output size
output_size = vocab_size

# Embedding Dimension
embedding_dim = 256

# Hidden Dimension
hidden_dim = 1024

# Number of RNN Layers
n_layers = 1

# Show stats for every n number of batches
show_every_n_batches = 1000

### Train
In the next cell, we'll train the neural network on the pre-processed data. If we have a hard time getting a good loss, we may consider changing your hyperparameters. In general, we may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **Let's aim for a loss less than 3.5.** 

We should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [18]:
# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout = 0.5)
if train_on_gpu: rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 4.655435397386551
Epoch:    1/10    Loss: 4.239656992912293
Epoch:    1/10    Loss: 4.130040341377258
Epoch:    2/10    Loss: 3.933150232802183
Epoch:    2/10    Loss: 3.8587693543434143
Epoch:    2/10    Loss: 3.863505454778671
Epoch:    3/10    Loss: 3.6609939495829558
Epoch:    3/10    Loss: 3.6118865551948547
Epoch:    3/10    Loss: 3.6450436782836912
Epoch:    4/10    Loss: 3.4577662226646093
Epoch:    4/10    Loss: 3.4206789886951445
Epoch:    4/10    Loss: 3.4465147030353545
Epoch:    5/10    Loss: 3.2526838925649537
Epoch:    5/10    Loss: 3.233358478307724
Epoch:    5/10    Loss: 3.297451806306839
Epoch:    6/10    Loss: 3.090326201080312
Epoch:    6/10    Loss: 3.073091745853424
Epoch:    6/10    Loss: 3.151703127145767
Epoch:    7/10    Loss: 2.9455674218360306
Epoch:    7/10    Loss: 2.9432005524635314
Epoch:    7/10    Loss: 3.020909842967987
Epoch:    8/10    Loss: 2.8240953208301267
Epoch:    8/10    Loss: 2.83473804426

### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** I started with exploring different settings for sequence length and batch size then realized that this didn't impact much on either the performance or convergence. Most of options I tried had a training loss of 5.X for batch 1 in epoch 1. I decided to look for research papers already published on the same, came across [paper](https://arxiv.org/ftp/arxiv/papers/1908/1908.04332.pdf), from which I tried LSTM single layer option with a `batch size, hidden_dim = 256, 1024`. With `sequence_length = 10` the loss for first `n=1000` batches was **4.85**. With `sequence_length = 20` then `sequence_length = 25`, convergence improved, next step was to modify embedding dimension from 200 to 256 and see if the convergence of loss improves, it did. Then added weight initialization method and the final selected model gave 2.74 training loss.

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [0]:
_, w2i, i2w, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, We'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. We'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [0]:
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len = 100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        if(train_on_gpu):
            current_seq = current_seq.cpu() # move to cpu
        # the generated word becomes the next "current sequence" and the cycle can continue
        if train_on_gpu:
            current_seq = current_seq.cpu()
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [21]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, w2i[prime_word + ':'], i2w, 
                            token_dict, w2i[pad_word], gen_length)
print(generated_script)

jerry: the way to the top of the muffin.

jerry: well, what do you mean?

elaine: i know, i know. it's not a good idea, you know, it's just the guy.

jerry: well, i guess i just had to tell you something. i don't know if i got the clothes in your apartment.

elaine: you know, if i can't get my coat, you know. you should be on your way up here.

jerry: oh, no, no...

kramer:(to jerry) oh, i don't know.....

susan: i just want to see that.

george: oh! oh!!!!

elaine: oh, i got a new coat.

jerry: oh, you have a problem with that?

george: i know.

kramer: you don't think i got the new catalog? i thought you were going to the movies.

jerry: oh, no thanks.

elaine: i can't believe you feel that.

jerry: oh, that's a good idea.(elaine puts it down on the back of his pocket.)

jerry: so you were going to get that appointment for me?

elaine: oh yeah. i don't like it.(they leave)

jerry: so what?

george: i know!

jerry: you know i was gonna get a new couch. you don't have to go to the movi

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [0]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 